In [14]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import optuna

df = pd.read_csv ('hw1_std.csv')

Clean & Prepare Data

In [15]:
df.fillna(value=0, inplace=True)

print(df)

     target  age sex   num1   num2   num3  num4  num5  ord1  ord2  ord3  ord4  \
0        -1   69   1  160.0  286.0  108.0   1.5   2.0   0.0   0.0   0.0   1.0   
1        -1   69   1  120.0  229.0  129.0   2.6   3.0   0.0   0.0   0.0   1.0   
2        -1   64   0  140.0  268.0  160.0   3.6   2.0   0.0   0.0   0.0   0.0   
3        -1   65   1  130.0  254.0  147.0   1.4   3.0   0.0   0.0   0.0   0.0   
4        -1   53   M  140.0  203.0  155.0   3.1   3.0   0.0   1.0   0.0   1.0   
..      ...  ...  ..    ...    ...    ...   ...   ...   ...   ...   ...   ...   
298       1   56   M  120.0  240.0  169.0   0.0   2.0   1.0   0.0   1.0   0.0   
299       1   55   M  132.0  342.0  166.0   1.2   2.0   1.0   0.0   1.0   0.0   
300       1   41   M  120.0  157.0  182.0   0.0   2.0   1.0   0.0   1.0   0.0   
301       1   38   M  138.0  175.0  173.0   0.0   2.0   2.0   0.0   1.0   0.0   
302       1   38   M  138.0  175.0  173.0   0.0   2.0   2.0   0.0   1.0   0.0   

     ord5  ord6  
0       1

In [16]:
df[df.duplicated (keep = False) == True]

df.drop_duplicates(keep = 'first')

target  age sex   num1   num2   num3  num4  num5  ord1  ord2  ord3  ord4  \
0        -1   69   1  160.0  286.0  108.0   1.5   2.0   0.0   0.0   0.0   1.0   
1        -1   69   1  120.0  229.0  129.0   2.6   3.0   0.0   0.0   0.0   1.0   
2        -1   64   0  140.0  268.0  160.0   3.6   2.0   0.0   0.0   0.0   0.0   
3        -1   65   1  130.0  254.0  147.0   1.4   3.0   0.0   0.0   0.0   0.0   
4        -1   53   M  140.0  203.0  155.0   3.1   3.0   0.0   1.0   0.0   1.0   
..      ...  ...  ..    ...    ...    ...   ...   ...   ...   ...   ...   ...   
297       1   56   M  130.0  221.0  163.0   0.0   3.0   1.0   0.0   0.0   0.0   
298       1   56   M  120.0  240.0  169.0   0.0   2.0   1.0   0.0   1.0   0.0   
299       1   55   M  132.0  342.0  166.0   1.2   2.0   1.0   0.0   1.0   0.0   
300       1   41   M  120.0  157.0  182.0   0.0   2.0   1.0   0.0   1.0   0.0   
301       1   38   M  138.0  175.0  173.0   0.0   2.0   2.0   0.0   1.0   0.0   

     ord5  ord6  
0       1     3  
1       1     2  
2       0     2  
3       1     1  
4       0     0  
..    ...   ...  
297     2     0  
298     0     0  
299     2     0  
300     2     0  
301     2     4  

[302 rows x 14 columns]

In [17]:
df['sex'] = df['sex'].replace({'F': '0'}) 
df['sex'] = df['sex'].replace({'M': '1'}) 

Feature Scaling

In [18]:
from sklearn.preprocessing import LabelEncoder

label_enc = LabelEncoder()
df['label'] = label_enc.fit_transform(df['target'])

In [19]:
data_df = df.drop(columns=['target'])
print(data_df)

     age sex   num1   num2   num3  num4  num5  ord1  ord2  ord3  ord4  ord5  \
0     69   1  160.0  286.0  108.0   1.5   2.0   0.0   0.0   0.0   1.0     1   
1     69   1  120.0  229.0  129.0   2.6   3.0   0.0   0.0   0.0   1.0     1   
2     64   0  140.0  268.0  160.0   3.6   2.0   0.0   0.0   0.0   0.0     0   
3     65   1  130.0  254.0  147.0   1.4   3.0   0.0   0.0   0.0   0.0     1   
4     53   1  140.0  203.0  155.0   3.1   3.0   0.0   1.0   0.0   1.0     0   
..   ...  ..    ...    ...    ...   ...   ...   ...   ...   ...   ...   ...   
298   56   1  120.0  240.0  169.0   0.0   2.0   1.0   0.0   1.0   0.0     0   
299   55   1  132.0  342.0  166.0   1.2   2.0   1.0   0.0   1.0   0.0     2   
300   41   1  120.0  157.0  182.0   0.0   2.0   1.0   0.0   1.0   0.0     2   
301   38   1  138.0  175.0  173.0   0.0   2.0   2.0   0.0   1.0   0.0     2   
302   38   1  138.0  175.0  173.0   0.0   2.0   2.0   0.0   1.0   0.0     2   

     ord6  label  
0       3      0  
1       2    

In [20]:
X = data_df.drop(columns=['label']).values
y = data_df['label'].values

In [21]:
feature_names = data_df.drop(columns=['label']).columns.values
print(feature_names)

['age' 'sex' 'num1' 'num2' 'num3' 'num4' 'num5' 'ord1' 'ord2' 'ord3'
 'ord4' 'ord5' 'ord6']


use 'OPTUNA'

In [46]:
def objective(trial):
    x = trial.suggest_uniform('x', -10, 10)
    return (x - 2) ** 2

study = optuna.create_study()
study.optimize(objective, n_trials = 100)

study.best_params

import sklearn.datasets
import sklearn.ensemble
import sklearn.model_selection
from sklearn.linear_model import LogisticRegression


def objective(trial):
    iris = sklearn.datasets.load_iris()
    x, y = iris.data, iris.target

    classifier_name = trial.suggest_categorical("classifier", ["RandomForest","LogReg"])
    
    if classifier_name == "RandomForest":
    
        rf_max_depth = trial.suggest_int("rf_max_depth", 2, 32, log=True)
        rf_n_estimators = trial.suggest_int(" rf_n_estimators", 10, 100)
        classifier_obj = sklearn.ensemble.RandomForestClassifier( max_depth = rf_max_depth, n_estimators = rf_n_estimators )
            

    else: 
        
        logReg_c = trial.suggest_float("logReg_c", 1e-10, 1e10, log = True)
        classifier_obj = LogisticRegression(C = logReg_c)
        
        
    score = sklearn.model_selection.cross_val_score(classifier_obj, x, y, n_jobs = -1, cv = 3)
    accuracy = score.mean()
    return accuracy


if __name__ == "__main__":
    study = optuna.create_study(direction="maximize")
    study.optimize(objective, n_trials = 100)
    print(study.best_trial)

[I 2021-03-08 21:48:17,121] A new study created in memory with name: no-name-2b63c880-2a28-4f1e-b583-5a4abef0a29a
[I 2021-03-08 21:48:17,125] Trial 0 finished with value: 38.32975014467374 and parameters: {'x': -4.1911024983175444}. Best is trial 0 with value: 38.32975014467374.
[I 2021-03-08 21:48:17,128] Trial 1 finished with value: 30.684694197795153 and parameters: {'x': 7.539376697589283}. Best is trial 1 with value: 30.684694197795153.
[I 2021-03-08 21:48:17,130] Trial 2 finished with value: 32.62169859820218 and parameters: {'x': 7.7115408252241515}. Best is trial 1 with value: 30.684694197795153.
[I 2021-03-08 21:48:17,132] Trial 3 finished with value: 86.92155228917467 and parameters: {'x': -7.323172865992278}. Best is trial 1 with value: 30.684694197795153.
[I 2021-03-08 21:48:17,134] Trial 4 finished with value: 72.2323920620941 and parameters: {'x': -6.498964175833082}. Best is trial 1 with value: 30.684694197795153.
[I 2021-03-08 21:48:17,137] Trial 5 finished with value: 

[I 2021-03-08 21:48:17,471] Trial 45 finished with value: 0.002955744818263532 and parameters: {'x': 2.0543667620726445}. Best is trial 30 with value: 5.777286146415907e-06.
[I 2021-03-08 21:48:17,479] Trial 46 finished with value: 20.87728890842182 and parameters: {'x': -2.5691672007513384}. Best is trial 30 with value: 5.777286146415907e-06.
[I 2021-03-08 21:48:17,488] Trial 47 finished with value: 1.2989901554321264 and parameters: {'x': 3.139732492926356}. Best is trial 30 with value: 5.777286146415907e-06.
[I 2021-03-08 21:48:17,496] Trial 48 finished with value: 126.80574977190858 and parameters: {'x': -9.260805911297316}. Best is trial 30 with value: 5.777286146415907e-06.
[I 2021-03-08 21:48:17,507] Trial 49 finished with value: 1.8965715112848993 and parameters: {'x': 0.6228393298946926}. Best is trial 30 with value: 5.777286146415907e-06.
[I 2021-03-08 21:48:17,519] Trial 50 finished with value: 0.0024290455242908585 and parameters: {'x': 2.049285347967635}. Best is trial 30 

[I 2021-03-08 21:48:17,910] Trial 90 finished with value: 1.5990451256261105 and parameters: {'x': 0.7354664395018571}. Best is trial 30 with value: 5.777286146415907e-06.
[I 2021-03-08 21:48:17,919] Trial 91 finished with value: 0.04509261093305797 and parameters: {'x': 1.7876497917753364}. Best is trial 30 with value: 5.777286146415907e-06.
[I 2021-03-08 21:48:17,928] Trial 92 finished with value: 0.3840701929995009 and parameters: {'x': 2.6197339695381405}. Best is trial 30 with value: 5.777286146415907e-06.
[I 2021-03-08 21:48:17,938] Trial 93 finished with value: 0.6443469096909558 and parameters: {'x': 1.1972877790322638}. Best is trial 30 with value: 5.777286146415907e-06.
[I 2021-03-08 21:48:17,948] Trial 94 finished with value: 0.03515750285338949 and parameters: {'x': 1.8124966590873925}. Best is trial 30 with value: 5.777286146415907e-06.
[I 2021-03-08 21:48:17,959] Trial 95 finished with value: 11.561941197848471 and parameters: {'x': -1.400285458288535}. Best is trial 30 w

[I 2021-03-08 21:48:26,796] Trial 30 finished with value: 0.9733333333333333 and parameters: {'classifier': 'LogReg', 'logReg_c': 6588.108284242874}. Best is trial 24 with value: 0.98.
[I 2021-03-08 21:48:26,891] Trial 31 finished with value: 0.9733333333333333 and parameters: {'classifier': 'LogReg', 'logReg_c': 45.77823143294439}. Best is trial 24 with value: 0.98.
[I 2021-03-08 21:48:26,983] Trial 32 finished with value: 0.9733333333333333 and parameters: {'classifier': 'LogReg', 'logReg_c': 982.3329933877185}. Best is trial 24 with value: 0.98.
[I 2021-03-08 21:48:27,093] Trial 33 finished with value: 0.9733333333333333 and parameters: {'classifier': 'LogReg', 'logReg_c': 179054.1696137799}. Best is trial 24 with value: 0.98.
[I 2021-03-08 21:48:27,183] Trial 34 finished with value: 0.9733333333333333 and parameters: {'classifier': 'LogReg', 'logReg_c': 10.795276928225435}. Best is trial 24 with value: 0.98.
[I 2021-03-08 21:48:27,226] Trial 35 finished with value: 0.87333333333333

[I 2021-03-08 21:48:30,933] Trial 73 finished with value: 0.9733333333333333 and parameters: {'classifier': 'LogReg', 'logReg_c': 21.127912907530785}. Best is trial 24 with value: 0.98.
[I 2021-03-08 21:48:31,039] Trial 74 finished with value: 0.9733333333333333 and parameters: {'classifier': 'LogReg', 'logReg_c': 86.00992595438349}. Best is trial 24 with value: 0.98.
[I 2021-03-08 21:48:31,127] Trial 75 finished with value: 0.9733333333333333 and parameters: {'classifier': 'LogReg', 'logReg_c': 3975.258090259612}. Best is trial 24 with value: 0.98.
[I 2021-03-08 21:48:31,206] Trial 76 finished with value: 0.9733333333333333 and parameters: {'classifier': 'LogReg', 'logReg_c': 0.7749288654787773}. Best is trial 24 with value: 0.98.
[I 2021-03-08 21:48:31,306] Trial 77 finished with value: 0.9733333333333333 and parameters: {'classifier': 'LogReg', 'logReg_c': 507.90321321201856}. Best is trial 24 with value: 0.98.
[I 2021-03-08 21:48:31,402] Trial 78 finished with value: 0.98 and param

FrozenTrial(number=88, values=[0.9866666666666667], datetime_start=datetime.datetime(2021, 3, 8, 21, 48, 32, 373344), datetime_complete=datetime.datetime(2021, 3, 8, 21, 48, 32, 484401), params={'classifier': 'LogReg', 'logReg_c': 8.266899750350351}, distributions={'classifier': CategoricalDistribution(choices=('RandomForest', 'LogReg')), 'logReg_c': LogUniformDistribution(high=10000000000.0, low=1e-10)}, user_attrs={}, system_attrs={}, intermediate_values={}, trial_id=88, state=TrialState.COMPLETE, value=None)


In [47]:
print(study.best_params)

{'classifier': 'LogReg', 'logReg_c': 8.266899750350351}


ANSWER

Q1. choose all attributes to become my features because it all have a value that can use to compute. 

Q2. use cross validation to evaluate a model because it is nesessary for "OPTUNA"

Q3. logistic regression, I come up with the hyperparameters for the model from an internet that which hyperparameters is nesessary for each model.

Q4. Accuracy because it's an easiest way